In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
"""
try:
  import cuml
except (ImportError, KeyError, ModuleNotFoundError):
  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
  !python rapidsai-csp-utils/colab/pip-install.py
"""


'\ntry:\n  import cuml\nexcept (ImportError, KeyError, ModuleNotFoundError):\n  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git\n  !python rapidsai-csp-utils/colab/pip-install.py\n'

In [3]:
# check if cuda is installed correctly
#!nvcc --version

In [4]:
# Mount google drive to colab and change to correct directory
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [5]:
# Try to import packages, if exception is thrown install dependencies and kill runtime
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23
try:
  #import optuna
  import sdmetrics
except (ImportError, KeyError, ModuleNotFoundError):
  ## code to install dependencies
  !pip install pycaret==3.0.0
  !pip install sdv==1.0.0
  #### Pf Measures
  !pip install sdmetrics
  !pip install kmodes
  !pip install pomegranate==0.14.8   #needed for sdmetrics BNLikelihood
  #### Models
  #!pip install optuna

  display('Stopping RUNTIME! Colaboratory will restart automatically. Please run again.')
  import os
  os.kill(os.getpid(), 9)

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [6]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step2-Model-CPU.ipynb

In [7]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step3-SDG.ipynb

In [8]:
"""

%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
# necessary imports for the section
from sdv.single_table import TVAESynthesizer as TVAE

import pickle
import pandas as pd
import os
import sys

sd_id = 'SD305Q10'
sd_iter = 7
sd_path = f'../data/synthetic/{sd_id}_{sd_iter}.csv'

df = pd.read_csv(sd_path)
sd_num = len(df)

for label in df['label'].unique():
    print(f"{label}: {df[df['label']==label]['label'].sum()}")

sdg = TVAE.load(f'../pickles/SDGs/{sd_id}_TVAE.pkl')

for i in range(10 + sd_iter):
  s_data = sdg.sample(num_rows=sd_num)


for label in s_data['label'].unique():
    print(f"{label}: {s_data[s_data['label']==label]['label'].sum()}")

s_data.to_csv(sd_path, index=False)
"""

'\n\n%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks\n# necessary imports for the section\nfrom sdv.single_table import TVAESynthesizer as TVAE\n\nimport pickle\nimport pandas as pd\nimport os\nimport sys\n\nsd_id = \'SD305Q10\'\nsd_iter = 7\nsd_path = f\'../data/synthetic/{sd_id}_{sd_iter}.csv\'\n\ndf = pd.read_csv(sd_path)\nsd_num = len(df)\n\nfor label in df[\'label\'].unique():\n    print(f"{label}: {df[df[\'label\']==label][\'label\'].sum()}")\n\nsdg = TVAE.load(f\'../pickles/SDGs/{sd_id}_TVAE.pkl\')\n\nfor i in range(10 + sd_iter):\n  s_data = sdg.sample(num_rows=sd_num)\n\n\nfor label in s_data[\'label\'].unique():\n    print(f"{label}: {s_data[s_data[\'label\']==label][\'label\'].sum()}")\n\ns_data.to_csv(sd_path, index=False)\n'

In [9]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step4-SD-Models-pycaret.ipynb

In [10]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step5-Measures_PF.ipynb

In [11]:
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks

import pandas as pd
import sys
import os
import re

from sdv.metadata import SingleTableMetadata

sys.path.append('../src')
from PF_metrics import compute_all_pf_measures
from utils import (getExperimentConfig,
                   getPicklesFromDir,
                   get_synthetic_filepaths_from_original_data_id)

config = getExperimentConfig()
folders = config['folders']

settings = getPicklesFromDir(folders['settings_dir'])

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks


In [12]:
run_dataset = ['D305']
run_quality = 'Q100'


# read file f it exists, if not-create dataframe
pf_measures_filepath = f"../data/result/{run_dataset[0]}{run_quality}_pf_measures.csv"


if os.path.exists(pf_measures_filepath):
    result_df = pd.read_csv(pf_measures_filepath)
else:
    result_df = pd.DataFrame(columns=['Dataset id'])

for dataset_setting in settings:

    if run_dataset is not None and dataset_setting['meta']['id'] not in run_dataset:
        # Checks if run_dataset contains dataset_id's
        # if it does, run the experiment only on specified datasets
        print(f"Skip pf for data id: {dataset_setting['meta']['id']}")
        continue

    original_data_id = dataset_setting['meta']['id']
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(original_data_id)
    # there is no synthetic datasets according to the Data id, skip and try next
    if synthetic_datasets == []:
        print(f"Did not find synthetic dataset files")
        continue

    original_data = pd.read_csv(folders['real_dir']+dataset_setting['meta']['filename'], dtype=dataset_setting['meta']['cols_dtype'])
    metadata = SingleTableMetadata().load_from_json(dataset_setting['meta']['meta_filepath']).to_dict()


    print(f"Data id: {original_data_id}")
    for sd_filename in synthetic_datasets:
        sd_id = os.path.splitext(sd_filename)[0]
        quality = re.findall('Q\d+', sd_id)[0]
        sd_path = folders['sd_dir']+sd_filename

        if quality != run_quality:
            continue

        # Check if the dataset has already been evaluated
        row_exists = ((result_df['Dataset id'] == sd_id)).any()
        if not row_exists:
            print(f"Computing PF on: {sd_id}")
            synthetic_data = pd.read_csv(sd_path)

            pf_measures = compute_all_pf_measures(original_data=original_data,
                                                  synthetic_data=synthetic_data,
                                                  dataset_meta=dataset_setting['meta'],
                                                  metadata=metadata,
                                                  SD_id=sd_id)
            pf_measures['Quality'] = quality
            pf_measures['Original dataset'] = original_data_id
            result_df = pd.concat([result_df, pf_measures], axis=0, ignore_index=True)
        else:
            print(f"{sd_id} already computed.")

        # save the results
        result_df.to_csv(pf_measures_filepath, index=False)

Skip pf for data id: D205
Skip pf for data id: D3
Data id: D305
SD305Q100_0 already computed.
SD305Q100_1 already computed.
SD305Q100_2 already computed.
SD305Q100_3 already computed.
SD305Q100_4 already computed.
SD305Q100_5 already computed.
Computing PF on: SD305Q100_6
Computing PF on: SD305Q100_7
Computing PF on: SD305Q100_8
Computing PF on: SD305Q100_9
Skip pf for data id: D410
Skip pf for data id: D0
Skip pf for data id: D1
Skip pf for data id: D405
